In [ ]:
test_semantic_model_url = 'https://raw.githubusercontent.com/djouallah/semantic_sql_testing/refs/heads/main/semantic_model.txt' 
questions_url           = 'https://raw.githubusercontent.com/djouallah/semantic_sql_testing/refs/heads/main/questions.json'
SF = 1
output_dir="/tmp/llm"

In [ ]:
import requests
import json
import duckdb
import pathlib
import time
import pandas as pd
import re
from   openai import AzureOpenAI, OpenAIError
import datetime
import os


# Generate Data

In [ ]:
if SF <1 :
 schema = f"{str(SF).replace('.', '_')}"
else :
 schema = f'DS{SF:02d}'
os.makedirs(output_dir, exist_ok=True)
db_path = output_dir +"/"+ schema +".duckdb"
if not pathlib.Path(db_path).exists():
    con = duckdb.connect(db_path)
    con.sql("SET memory_limit = '14GB' ")
    con.sql(f"CALL dsdgen(sf={SF})")
    con.close()
con = duckdb.connect()
con.sql(f""" attach '{db_path}' as ds(read_only) ; use ds """)
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# Run Tests

In [ ]:
def get_ai_response(user_message, LLM, provider, api_key=None, azure_endpoint=None, azure_api_version=None):
    system_prompt = ""
    try:
        github_response = requests.get(test_semantic_model_url)
        github_response.raise_for_status()
        system_prompt = github_response.text.strip()
    except requests.RequestException as e:
        print(f"Error fetching system prompt from GitHub ({test_semantic_model_url}): {e}")
        return f"Error fetching system prompt: {e}"

    generated_text = None
    data = None

    if provider == "ollama":
        url = 'http://127.0.0.1:11434/api/chat'
        headers = {'Content-Type': 'application/json'}
        payload = {
            'model': LLM,
            'messages': [
                {'role': 'system', 'content': system_prompt},
                {'role': 'user', 'content': user_message}
            ],
            'stream': False
        }
        try:
            response = requests.post(url, headers=headers, json=payload)
            response.raise_for_status()
            data = response.json()
            generated_text = data.get('message', {}).get('content', '')
            if generated_text:
                generated_text = generated_text.replace("<think>", "").replace("</think>", "")
        except requests.RequestException as e:
            return f"Error with Ollama API request: {e}"
        except (KeyError, IndexError, AttributeError) as e:
            return f"Unexpected response format from Ollama: {e}\nFull response data: {data}"

    elif provider == "azure_foundry":
        if not azure_endpoint or not azure_api_version or not api_key:
            return "Azure provider requires 'azure_endpoint', 'azure_api_version', and 'api_key' parameters."

        try:
            client = AzureOpenAI(
                api_version=azure_api_version,
                azure_endpoint=azure_endpoint,
                api_key=api_key,
            )

            response = client.chat.completions.create(
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_message}
                ],
                model=LLM,
            )

            generated_text = response.choices[0].message.content

        except Exception as e:
            return f"An error occurred with Azure OpenAI: {e}"



    else:
        return f"Unsupported provider for this test setup: {provider}. Supported providers are 'ollama', 'azure_foundry', and 'gemini'."

    if not isinstance(generated_text, str):
         print(f"Generated text is not a string (type: {type(generated_text)}). Cannot clean.")
         if generated_text is None:
              return "Could not retrieve generated text from provider response."
         else:
              return f"Received unexpected output type from {provider}: {type(generated_text)}. Value: {generated_text}"


    cleaned_text = re.sub(r'```(sql|duckdb)?\s*([\s\S]*?)\s*```', r'\2', generated_text, flags=re.IGNORECASE).strip()

    if "```" in cleaned_text or "SELECT " in cleaned_text.upper() or "WITH " in cleaned_text.upper():
         cleaned_text = cleaned_text.replace("```sql", "").replace("```duckdb", "").replace("```", "").strip()


    return cleaned_text

In [ ]:
def execute_sql_with_retry(query, test_model, provider, api_key, azure_endpoint, azure_api_version, max_attempts):
    attempt = 1
    initial_query = query.strip()
    current_query = initial_query

    while attempt <= max_attempts:
        try:
            result = con.sql(current_query).df()
            query_returned = initial_query if attempt == 1 else current_query
            return result, attempt, query_returned
        except Exception as e:
            error_message = str(e).lower()

            if "syntax" not in error_message and "parser" not in error_message and "binder" not in error_message:
                return f"Non-syntax error: {e}", attempt, current_query

            print(current_query)
            print(f"Attempt {attempt}/{max_attempts} failed with syntax error: {e}")

            if attempt == max_attempts:
                return f"Max attempts reached. Last error: {e}", attempt, current_query

            message = (
                f"The following SQL query has a syntax error: '{current_query}'.\n"
                f"Error message: {e}\n"
                f"Please provide the corrected SQL query. Return only the corrected query without explanation."
            )

            corrected_query = get_ai_response(message, test_model, provider, api_key, azure_endpoint, azure_api_version)

            if corrected_query.startswith("Error"):
                return f"Failed to get corrected query : {corrected_query}", attempt, current_query

            current_query = corrected_query.strip()
            attempt += 1

    return "Unexpected error or loop termination", attempt, current_query

In [ ]:
def ask_question(questions, test_model,provider,api_key=None, azure_endpoint=None, azure_api_version=None):
    results_data = [] 
    for i, x in enumerate(questions):
        print(f"Question {i+1}: {x}") # Keep or remove print as needed
        start_time = time.time()
        sql_query_or_error = get_ai_response(x, test_model, provider, api_key, azure_endpoint, azure_api_version)
        print(sql_query_or_error) # Keep or remove print as needed
        query_result_data_json = [] # Initialize as an empty list for JSON result
        attempts_count = None
        error_details = None # Initialize error_details
        if sql_query_or_error is None or sql_query_or_error.startswith("Error"):
            # If get_ai_response returned an error or None, store the error string
            error_message = sql_query_or_error if sql_query_or_error is not None else "AI response was None"
            # print(f"Failed to get query from AI: {error_message}") # Keep or remove print
            # print("Execution: SKIPPED (AI error)") # Keep or remove print
            error_details = f"AI Error: {error_message}" # Store error details
            # query_result_data_json remains empty []
            result_row_count = 0 # Result count is 0 on error
        else:
            result_from_execution, attempts_count,query_returned = execute_sql_with_retry(sql_query_or_error, test_model,provider,api_key, azure_endpoint, azure_api_version, max_attempts=3)

            display(result_from_execution)
            is_successful = isinstance(result_from_execution, pd.DataFrame)

            if is_successful:
                print("Execution: SUCCESS") # Keep or remove print
                query_result_data_json = result_from_execution.to_dict('records')
                error_details = None # No error details on success
                result_row_count = len(result_from_execution) # Calculate row count
            else:
                print("Execution: FAILED") # Keep or remove print
                #query_result_data_json remains empty []
                error_details = f"Execution Error: {result_from_execution}" # Store error details
                result_row_count = 0 # Result count is 0 on failure


        end_time = time.time()
        duration = round(end_time - start_time, 2)
        print(f"\nExecution Time: {duration:.2f} seconds") # Keep or remove print
        print(f" ############################### ") # Keep or remove print
        results_data.append({
            "model" : test_model,
            "timestamp": timestamp,
            "nbr": i + 1,
            "question": x,
            "duration_s": duration, 
            "sql_query": query_returned, 
            "attempts": attempts_count,
            "result": query_result_data_json, 
            "result_count": result_row_count, 
            "error_details": error_details 
        })

    os.makedirs(output_dir, exist_ok=True)
    sanitized_model = re.sub(r'[\\/*?:"<>|]', '_', test_model)
    output_filename = f"{timestamp}_{sanitized_model}.json"
    output_path = os.path.join(output_dir, output_filename)
    try:
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(results_data, f, indent=4)
        return f"Successfully processed {len(questions)} questions. Results saved to {output_path}"
    except IOError as e:
        return f"Error saving results to {output_path}: {e}"
    except Exception as e:
        return f"An unexpected error occurred during file saving: {e}"

In [ ]:
try:
    response = requests.get(questions_url)
    response.raise_for_status()  # Raise an exception for HTTP errors (4xx or 5xx)
    questions = json.loads(response.text)

    print("Successfully retrieved questions:")
except requests.exceptions.RequestException as e:
    print(f"Error retrieving file from GitHub: {e}")
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")

In [ ]:
%%time
#ask_question(questions,"o3-mini","azure_foundry",os.getenv("o3_mini_key"), os.getenv("llm_endpoint"),"2025-01-01-preview" )

In [ ]:
%%time
#ask_question(questions,"gpt-4o","azure_foundry",os.getenv("LLM_API_KEY"), os.getenv("llm_endpoint"),"2025-01-01-preview" )

In [ ]:
%%time
ask_question(questions,'qwen3:8b',"ollama")

# Check Results

In [41]:
duckdb.sql(f""" from '{output_dir}/*.json' order by nbr,model """).to_view("results")
try:
 duckdb.sql(f""" load excel ;copy results to '{output_dir}/test.xlsx' (format 'xlsx', header 'true', overwrite)  """)
except Exception as e:
    print(f"Error exporting to Excel: {e}")
    print("close the excel file or tey again.")

Error exporting to Excel: IO Error: Could not move file: The process cannot access the file because it is being used by another process.

close the excel file or tey again.


In [ ]:
# check number of rows and column returned by each model, Null means SQL error
duckdb.sql(f""" 
           pivot( select nbr,question,model,[result_count , cardinality(result[1])] as result_shape from results ) 
           on model using min(result_shape) order by nbr
        """).show(max_width=150)

┌───────┬───────────────────────────────────────────────────────────────────────────────────────────┬───────────┬───────────┬────────────┬───────────┐
│  nbr  │                                         question                                          │  o3-mini  │ qwen3:14b │  qwen3:4b  │ qwen3:8b  │
│ int64 │                                          varchar                                          │ int128[]  │ int128[]  │  int128[]  │ int128[]  │
├───────┼───────────────────────────────────────────────────────────────────────────────────────────┼───────────┼───────────┼────────────┼───────────┤
│     1 │ What is the overall total sales revenue?                                                  │ [1, 1]    │ [1, 1]    │ [1, 1]     │ [1, 1]    │
│     2 │ What is the total number of items sold across all transactions?                           │ [1, 1]    │ [1, 1]    │ [1, 1]     │ [1, 1]    │
│     3 │ What is the total monetary value of all returned items?                             

In [ ]:
# execusion time in seconds
duckdb.sql(f""" 
           pivot( select nbr,question,model,duration_s from results ) 
           on model using min(duration_s) as duration order by nbr
        """).show(max_width=150)

┌───────┬────────────────────────────────────────────────────────────┬──────────────────┬────────────────────┬───────────────────┬───────────────────┐
│  nbr  │                          question                          │ o3-mini_duration │ qwen3:14b_duration │ qwen3:4b_duration │ qwen3:8b_duration │
│ int64 │                          varchar                           │      double      │       double       │      double       │      double       │
├───────┼────────────────────────────────────────────────────────────┼──────────────────┼────────────────────┼───────────────────┼───────────────────┤
│     1 │ What is the overall total sales revenue?                   │             4.75 │              52.04 │             14.41 │             27.56 │
│     2 │ What is the total number of items sold across all transa…  │             4.43 │              15.09 │              1.91 │              9.25 │
│     3 │ What is the total monetary value of all returned items?    │             3.72 │     

Manual check for smilar results ( same rows and columns)

In [ ]:
manual = duckdb.sql("""
   with   zzzzz AS(
    SELECT 
        nbr, 
        model, 
        result
    FROM results 
    WHERE
    model in ('o3-mini','qwen3:8b') and 
    nbr not in
    (
    select nbr from results 
    where model in ('o3-mini','qwen3:8b')
    group by all
    having count(distinct(cardinality(result[1]))) > 
    1 or count(distinct(result_count )) > 1 order by nbr)
    )
    
pivot(from zzzzz ) on model using min(result) order by nbr
""")
manual.show(max_width=250)
try:
 duckdb.sql(f""" load excel ;copy manual to '{output_dir}/results_similar.xlsx' (format 'xlsx', header 'true', overwrite)  """)
except Exception as e:
    print(f"Error exporting to Excel: {e}")
    print("close the excel file or tey again.")

┌───────┬──────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│  nbr  │       o3-mini        │                                                                                                        qwen3:8b                                                                                                         │
│ int64 │ map(varchar, json)[] │                                                                                                  map(varchar, json)[]                                                                                                   │
├───────┼──────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [49]:
different_results = duckdb.sql("""
    SELECT 
        distinct(nbr) 
    FROM results 
    WHERE
    model in ('o3-mini','qwen3:8b') and 
    nbr in
    (
    select nbr from results 
    where model in ('o3-mini','qwen3:8b')
    group by all
    having count(distinct(cardinality(result[1]))) > 
    1 or count(distinct(result_count )) > 1 )
    order by nbr
""").df()['nbr'].tolist()
for nbr in different_results:
    print(f"question {nbr} : " + duckdb.sql(f" select question from results where nbr = {nbr} and model in ('o3-mini') ").fetchone()[0])
    try:
        query = duckdb.sql(f" select sql_query from results where nbr = {nbr} and model in ('o3-mini') ").fetchone()[0]
        df1 = con.sql(query).df()
        query = duckdb.sql(f" select min(sql_query) from results where nbr = {nbr} and model in ('qwen3:8b') ").fetchone()[0]
        df2 = con.sql(query).df()
        side_by_side = pd.concat([df1, df2], axis=1, keys=['DF1', 'DF2'])
        display(side_by_side)
    except Exception as e:
        print(f"Error executing query for nbr {nbr}: {e}")

question 7 : Which store generated the most total sales revenue? (To ensure a deterministic result if there's a tie, you might order by revenue descending and then store name alphabetically).


DF1                        DF2              
  s_store_name   total_sales s_store_name   total_sales
0          bar  8.485581e+08          bar  8.485581e+08
1          NaN           NaN         eing  8.471117e+08
2          NaN           NaN          ese  8.462294e+08
3          NaN           NaN         able  8.459570e+08
4          NaN           NaN        ation  8.452357e+08
5          NaN           NaN        ought  8.437834e+08

question 14 : Which customer birth country exhibits the highest average return rate Order by average return rate descending ?


DF1                               DF2                \
        birth_country avg_return_rate     birth_country   total_sales   
0                FIJI        5.952939              FIJI  2.384176e+07   
1            BARBADOS        5.921375          BARBADOS  2.270181e+07   
2    CHRISTMAS ISLAND        5.919433  CHRISTMAS ISLAND  2.431222e+07   
3            ANGUILLA        5.828528          ANGUILLA  2.360135e+07   
4           SRI LANKA        5.826083         SRI LANKA  2.504351e+07   
..                ...             ...               ...           ...   
208            GAMBIA        4.675416            GAMBIA  2.283661e+07   
209              OMAN        4.610193              OMAN  2.359265e+07   
210         GIBRALTAR        4.496520         GIBRALTAR  2.371106e+07   
211              None        0.000000              None  1.738668e+08   
212              None             NaN              None  0.000000e+00   

                               
    total_returns return_rate  
0      1419285.49    5.952939  
1      1344259.12    5.921375  
2      1439145.65    5.919433  
3      1375611.61    5.828528  
4      1459055.80    5.826083  
..            ...         ...  
208    1067706.60    4.675416  
209    1087666.67    4.610193  
210    1066172.79    4.496520  
211          0.00    0.000000  
212    9037616.82         NaN  

[213 rows x 6 columns]

question 15 : List all item product names that have a return rate greater than 5% and their total sales, ordered by item product name alphabetically.


...

question 17 : What is the return rate for items sold on weekends versus weekdays, broken down by customer age groups (e.g., under 30, 30-45, over 45), ordered by age group.


DF1                                                             DF2  \
   age_group sale_day_type   total_sales total_returns return_rate age_group   
0      30-45       Weekday  8.036384e+08   42231793.68    5.255074     20-29   
1      30-45       Weekend  3.256532e+08   16958203.53    5.207442     20-29   
2    over 45       Weekday  1.691612e+09   89372705.33    5.283286     30-39   
3    over 45       Weekend  6.773807e+08   35478913.87    5.237663     30-39   
4   under 30       Weekday  1.106583e+09   56395803.78    5.096389     40-49   
5   under 30       Weekend  4.411807e+08   23060101.47    5.226906     40-49   
6        NaN           NaN           NaN           NaN         NaN     50-59   
7        NaN           NaN           NaN           NaN         NaN     50-59   
8        NaN           NaN           NaN           NaN         NaN       60+   
9        NaN           NaN           NaN           NaN         NaN       60+   
10       NaN           NaN           NaN           NaN         NaN      < 20   
11       NaN           NaN           NaN           NaN         NaN      < 20   

                                                    
   weekday   total_sales total_returns return_rate  
0        N  5.098070e+08   26393168.04    5.177090  
1        Y  2.025206e+08   11042123.56    5.452347  
2        N  5.036592e+08   26550551.04    5.271531  
3        Y  2.044319e+08   10821169.42    5.293287  
4        Y  2.025687e+08   10441801.82    5.154697  
5        N  5.053419e+08   26399997.04    5.224185  
6        Y  2.029946e+08   10674998.32    5.258759  
7        N  5.058094e+08   26516422.79    5.242374  
8        Y  3.410808e+08   17956562.10    5.264606  
9        N  8.573134e+08   45801470.62    5.342442  
10       Y  2.386602e+08   12017977.91    5.035602  
11       N  5.967765e+08   30002635.74    5.027449

question 18 : Which item brand has shown the largest decrease in its return rate when comparing the average rate of 2001 to 2002, specifically for stores located in the 'TN' state? (Order by the decrease in return rate descending, and then by brand name alphabetically for ties).


DF1                                            DF2  \
                   brand decrease_in_return_rate                i_brand   
0    exportiunivamalg #9               20.812102  scholaramalgamalg #11   
1     amalgamalgamalg #7               17.018305     corpamalgamalg #12   
2       corpunivamalg #3               15.805714    exportiunivamalg #4   
3     amalgamalgamalg #5               15.586534     univamalgamalg #10   
4       maxiunivamalg #9               15.579876       namelessbrand #7   
..                   ...                     ...                    ...   
693                  NaN                     NaN    importounivamalg #7   
694                  NaN                     NaN  scholaramalgamalg #17   
695                  NaN                     NaN       maxiunivamalg #4   
696                  NaN                     NaN  scholaramalgamalg #16   
697                  NaN                     NaN      corpunivamalg #16   

                          
    return_rate_decrease  
0              871129.00  
1              829303.40  
2              815780.87  
3              670829.37  
4              620705.92  
..                   ...  
693           -787329.83  
694           -808603.67  
695           -948373.00  
696           -957959.50  
697          -1074398.83  

[698 rows x 4 columns]

question 20 : show all stores net sales , average net sales and percentage of net sales compared to average net sales for each store, ordered by percentage of net sales descending.
Error executing query for nbr 20: Parser Error: syntax error at or near ")"
